In [72]:
import re
import time


class Sensor:
    def __init__(self, sensor_coords, beacon_coords):
        self.sensor_coords = sensor_coords
        self.beacon_coords = beacon_coords
        
    def get_distance(self, coord=None):
        x1, y1 = self.sensor_coords
        x2, y2 = coord
        return abs(x1 - x2) + abs(y1 - y2)
    
    def in_range(self, coord=None):
        distance = self.get_distance(coord)
        max_distance = self.get_max_distance()
        return distance <= max_distance
    
    def get_max_distance(self):
        return self.get_distance(self.beacon_coords)
    
    def get_manhattan_neighborhood(self):
        neighborhood = set()
        x, y = self.sensor_coords
        d = self.get_max_distance()

        # Loop over all points in the neighborhood
        for i in range(x - d, x + d + 1):
            for j in range(y - d, y + d + 1):
              # Calculate the Manhattan distance between the given point and the current point
              distance = abs(x - i) + abs(y - j)

              # If the distance is less than or equal to d, add the point to the neighborhood
              if distance <= d:
                neighborhood.add((i, j))

        return neighborhood
    
    def get_bounds(self):
        x, y = self.sensor_coords
        d = self.get_max_distance()
        
        min_x, min_y, max_x, max_y = x-d, y-d, x+d, y+d
        
        return min_x, min_y, max_x, max_y
    
    def get_coordinates_in_row(self, y, min_x=float('-inf'), max_x=float('inf')):
        neighborhood = set()
        x = self.sensor_coords[0]
        d = self.get_max_distance()
        
        coordinates = set()
        if y >= (y-d) or y <= (y + d):
            
            lb = max(min_x, x - d)
            ub = min(max_x, x+d)
            r = range(lb, ub+1)
            
            for x in r:
                distance = self.get_distance((x, y))
                if distance <= d:
                    coordinates.add((x, y))
        else:
            print('skip', self)
                
        return coordinates
    

    
    def get_row_range(self, y, min_x=float('-inf'), max_x=float('inf')):
        x1, y1 = self.sensor_coords
        y2 = y
        d = self.get_max_distance()
        
        u = d - abs(y1 - y2)
        x2lb = x1 - u
        x2ub = x1 + u
        
        r = None
        if x2lb < x2ub:
            r = [max(min_x, x2lb), min(x2ub, max_x)]
                
        return r



    def __str__(self):
        return f"s: {self.sensor_coords} - d: {self.get_max_distance()}"
    
    def __repr__(self):
        return self.__str__()
    
    
def merge(intervals):

    if len(intervals) <= 1:
        return intervals

    intervals = sorted(intervals, key=lambda x: x[0])
    merged_intervals = []
    idx = 0
    while idx < len(intervals):
        curr = intervals[idx]
        if not merged_intervals or merged_intervals[-1][1] < curr[0]:
            merged_intervals.append(curr)
        else:
            merged_intervals[-1][1] = max(curr[1], merged_intervals[-1][1])

        idx += 1

    return merged_intervals
    
def get_coordinates_where_beacon_not_present(sensors, y, min_x, max_x):
    coords = set()
    for sensor in sensors:
        neighborhood = sensor.get_coordinates_in_row(y, min_x, max_x)
        for coord in neighborhood:
            if coord[1] == y and coord != sensor.beacon_coords:
                coords.add(coord)
    return coords

def get_range_where_beacon_not_present(sensors, y, min_x, max_x):
    ranges = []
    for sensor in sensors:
        r = sensor.get_row_range(y, min_x, max_x)
#         print(sensor, r)
        if r:
            ranges.append(r)
    return merge(ranges)



with open('../inputs/15.txt') as f:
    sensors = []
    for line in f:
        result = re.search(r"Sensor at x=(.*), y=(.*): closest beacon is at x=(.*), y=(.*)", line)
        groups = map(int, result.groups())
        sx1, sy1, bx1, by1 = groups

        sensor = Sensor((sx1, sy1), (bx1, by1))
        sensors.append(sensor)

    size = 4000000


    s = time.time()
    for i in range(size):
        range_ = get_range_where_beacon_not_present(sensors, i, 0, size)
        if i % 100000 == 0:
            print(i)
        if len(range_) > 1:
            print(i, range_)
            y = i
            x = range_[0][1] + 1
            result = 4000000 * x + y
            print(result, time.time() - s)
            break
        



0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2686239 [[0, 3316867], [3316869, 4000000]]
13267474686239 45.98431324958801


In [70]:
y = 2686239
x = 3316867 + 1

result = 4000000 * x + y
print(result)

13267474686239
